In [ ]:
import pandas as pd
import json
import os
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from datetime import date
from dateutil import parser
from dateutil.relativedelta import relativedelta

In [ ]:
class TradeResult(object):
    def __init__(self, enter_date, enter_ticker, enter_price, initial_stop, target_price, enter_direction, exit_date, exit_price, is_missed):
        self.enter_date = enter_date
        self.enter_ticker = enter_ticker
        self.enter_price = enter_price
        self.initial_stop = initial_stop
        self.target_price = target_price
        self.enter_direction = enter_direction
        self.exit_date = exit_date
        self.exit_price = exit_price
        self.is_missed = is_missed
    def todf(self):
        return pd.DataFrame({
                "EnterDate": self.enter_date,
                "EnterTicker": self.enter_ticker,
                "EnterPrice": self.enter_price, 
                "InitialStop": self.initial_stop,
                "TargetPrice": self.target_price,
                "EnterDirection": self.enter_direction,
                "ExitDate": self.exit_date,
                "ExitPrice": self.exit_price,
                "IsMissed": self.is_missed
            }, index=[0])
    def tostr(self):
        print(str.format(
                "EnterDate: {0} \tEnterTicker: {1}\nEnterPrice: {2} \t\tInitialStop: {3}\nTargetPrice: {4}\tEnterDirection: {5}\nExitDate: {6}\tExitPrice: {7} IsMissed: {8}"
                ,self.enter_date
                ,self.enter_ticker
                ,self.enter_price
                ,self.initial_stop
                ,self.target_price
                ,self.enter_direction
                ,self.exit_date
                ,self.exit_price
                ,self.is_missed
            ) )
    
def addyears(date_str, num):
    d = parser.parse(date_str).date()
    return str(d + relativedelta(years=num))
def adddays(date_str, num):
    d = parser.parse(date_str).date()
    return str(d + relativedelta(days=num))
def remove_candidate_if_traded(df, result_df):
    if any(trade_df.EnterTicker == df.iloc[0].Ticker) and trade_df.loc[trade_df.EnterTicker == df.iloc[0].Ticker].ExitDate.max() > df.iloc[0].Date:
        df.drop(df.head(1).index, inplace=True)
        remove_candidate_if_traded(df, result_df)

In [ ]:
#target_file = "C:\\temp\\TripleScreenBear.csv"
target_file = "C:\\temp\\WorkingInProgress.csv"
df = pd.read_csv(target_file, parse_dates=True)
#df = df.sort_index(ascending=False)

In [ ]:
trade_df = pd.DataFrame()

In [ ]:
# Login
driver = webdriver.Firefox()
driver.implicitly_wait(10) # seconds
driver.get("http://stockcharts.com")
account_json = 'C:\\Tools\\stockchart.json'
with open(account_json) as data_file:    
    account_info = json.load(data_file)
driver.find_element_by_link_text('Login').click()
driver.find_element_by_id('form_UserID').send_keys(account_info['user_name'])
driver.find_element_by_name('form_UserPassword').send_keys(account_info['password'])
driver.find_element_by_id('loginbutton').submit()

In [ ]:
ticker = df.iloc[0].Ticker
start_period = df.iloc[0].Date
one_year_back = addyears(start_period, -1)
driver.find_element_by_id('navsearchtext').send_keys(Keys.CONTROL + 'a' + Keys.NULL,ticker)
driver.find_element_by_id('navsearchbutton').click()
select = Select(driver.find_element_by_id('dataRange'))
select.select_by_visible_text('Select Start/End')
driver.find_element_by_id('start').send_keys(Keys.CONTROL + 'a' + Keys.NULL,one_year_back)
driver.find_element_by_id('submitButton').click() # otherwise the end date wont change
driver.find_element_by_id('end').send_keys(Keys.CONTROL + 'a' + Keys.NULL, start_period)
driver.find_element_by_id('submitButton').click()
print(str.format('Ticker: {0}\nDate: {1}', ticker, start_period))

In [ ]:
# Happy with the setup and move to the next records that is not traded
df.drop(df.head(1).index, inplace=True)
remove_candidate_if_traded(df, trade_df)
print(df.iloc[0].Ticker + ' ' + df.iloc[0].Date)

In [ ]:
enter_date = driver.find_element_by_id('end').get_attribute('value')
enter_ticker = ticker
enter_price = 45.0
initial_stop = 45.67
target_price = 43.2
enter_direction = 'Short'

In [ ]:
exit_date = driver.find_element_by_id('end').get_attribute('value')
exit_price = 42.7
is_missed = 0
trade = TradeResult(enter_date, enter_ticker, enter_price, initial_stop, target_price, enter_direction, exit_date, exit_price, is_missed)
trade.tostr()

In [ ]:
trade_df = trade_df.append(trade.todf(), ignore_index=True)
trade_df.tail(5)

In [ ]:
# save all the trades info for further documentation
# header = EnterDate,EnterDirection,EnterPrice,EnterTicker,ExitDate,ExitPrice,InitialStop,IsMissed,TargetPrice
result_file = "C:\\temp\\trade_result.csv"
with open(result_file, 'a+') as f:
    trade_df.to_csv(f, header=False, index=False)

In [ ]:
# save the records for next time to work
df.to_csv('C:\\temp\\WorkingInProgress.csv', index=False)